In [1]:
import numpy as np
from mne.decoding import CSP
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from util2 import A3Load
DIR, loc, EEG_eval, EEGL_eval, EEGR_eval, EEGL_train, EEGR_train = A3Load()

FileNotFoundError: [Errno 2] No such file or directory: './data/tr_EEGL_2.pkl'

In [ ]:
# need to format the data for MNE, ugh
I_PARTICIPANT = 6
I_FILTER = 1

N_EPOCHS = 72
N_CHANNELS = 22

l_train = []
r_train = []
for i in range(N_EPOCHS):
    l_train.append(EEGL_train[I_PARTICIPANT][I_FILTER][:,:,i])
    r_train.append(EEGR_train[I_PARTICIPANT][I_FILTER][:,:,i])

X = np.array(l_train + r_train)
X = X.astype(np.float64)

y_l = np.zeros(len(l_train))
y_r = np.ones(len(r_train))

y = np.concatenate((y_l, y_r))

X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize CSP
csp = CSP(n_components=4, norm_trace=False)

# Fit CSP
csp.fit(X, y)

In [ ]:
'''
Right/Left classifier
'''
# CSP transformattion on training data

X_filt_train = csp.transform(X_train)

# Setting up LDA for training
LDA = LinearDiscriminantAnalysis()

LDA.fit(X_filt_train, y_train)


# CSP transformattion on testing data
X_filt_test = csp.transform(X_test)

#predictions = LDA.predict(X_filt_test)
#acc = LDA.score(X_filt_test, y_test)
predictions = LDA.predict(X_filt_test)
acc = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print(f"accuracy: {acc} \n precision: {precision} \n recall: {recall} \n f1: {f1}")